[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# La extensión ```flask-restful```.

https://flask-restful.readthedocs.io/en/latest/

In [ ]:
!pip install flask-restful

In [ ]:
from flask import Flask, request, abort
from data import carreras, ruta, esquema_alumno
from json import loads, dumps
import jsonschema
from flask_restful import Api, Resource

In [ ]:
app = Flask(__name__)
api = Api(app)

In [ ]:
def accede_base(ruta, cuenta):
    alumno = None
    with open(ruta, 'tr') as f:
        base = eval(f.read())
    for item in base:
        try:
            if item['Cuenta'] == int(cuenta):
                alumno = item
        except:
            break
    return base, alumno

In [ ]:
def recurso_completo(base, ruta, cuenta, peticion):
    try:
        candidato = {'Cuenta': int(cuenta)}
        peticion = loads(peticion)
        jsonschema.validate(peticion, esquema_alumno)
        if set(esquema_alumno["properties"]).issuperset(peticion) \
        and peticion["Carrera"] in carreras:
            candidato.update(peticion)
        else:
            raise(TypeError, "Not valid")
    except (jsonschema.exceptions.ValidationError, TypeError):
        abort(406)
    base.append(candidato)
    with open(ruta, 'wt') as f:
            f.write(str(base))
    return candidato

In [ ]:
class Alumnos(Resource):
    
    
    def get(self):
        with open(ruta, 'tr') as base:    
            return eval(base.read())

In [ ]:
class ABCAlumno(Resource):
       
        
    def get(self, cuenta):
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            return alumno
        else:
            abort(404)
    
    
    def post(self, cuenta):
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            abort(409)
        else:
            return recurso_completo(base, ruta, cuenta, request.data)
    
    
    def delete(self, cuenta):
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            base.remove(alumno)
            with open(ruta, 'wt') as f:
                f.write(str(base))
            return alumno
        else:
            abort(404)
            
            
    def put(self, cuenta):
        base, alumno = accede_base(ruta, cuenta)
        if not alumno:
            abort(404)
        else:
            base.remove(alumno)
            return recurso_completo(base, ruta, cuenta, request.data)

        
    def patch(self, cuenta):
        base, alumno = accede_base(ruta, cuenta)
        if not alumno:
            abort(404)
        else:
            base.remove(alumno)
            del alumno['Cuenta']
            alumno.update(loads(request.data))
            return recurso_completo(base, ruta, cuenta, dumps(alumno))

In [ ]:
api.add_resource(Alumnos, '/api/')

In [ ]:
api.add_resource(ABCAlumno, '/api/<string:cuenta>')

In [ ]:
app.run(host='0.0.0.0')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>